## Imports for the Project and Setup

In [1]:
# Full reset of the camera
!echo 'dlinano' | sudo -S systemctl restart nvargus-daemon && printf '\n'
# Check device number
#!ls -ltrh /dev/video*

[sudo] password for dlinano: 


In [2]:
import torch
import torchvision
#from torchvision import transforms , datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary # for model summary
import matplotlib.pyplot as plt # for ploting our data and showing images
import matplotlib.image as mpimg
import numpy as np
import cv2
import dataHelper as dH
import os
import time
import random
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jetcam.csi_camera import CSICamera
%matplotlib inline

In [3]:
camera = CSICamera(width=224, height=224)

## Model Setup

Here we are letting pytorch know to use the gpu and we are also importing a pretrained resnet18 model that we are modifying to return only 2 outputs. One for X and one for Y.

In [4]:
device = torch.device('cuda')
output_dim = 2
# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, output_dim)
model = model.to(device)

## Model Summary
This is the inner workings of resnet18.

In [5]:
summary(model, input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

## Data Collection

### Files Structure Setup

In [6]:
path_raw = os.getcwd() + '/raw_datasets'
try:
    os.mkdir(path_raw)
except OSError:
    pass
else:
    pass

In [7]:
#config
datacollect = False
dataSetSize = 25
FPScollect = 30
SEC_delay = 3


loopcounter = 0
collectedNum = 0

if datacollect == True:
    image_widget = ipywidgets.Image(format='jpeg')
    captured_frame = ipywidgets.Image(format='jpeg')
    
    camera.unobserve_all()
    print('Get Ready!')
    frame = camera.read()
    image_widget.value = bgr8_to_jpeg(frame)
    captured_frame.value = bgr8_to_jpeg(frame)
    #plt.imshow(bgr8_to_jpeg(frame))
    time.sleep(6)
    display(ipywidgets.HBox([image_widget, captured_frame]))
    print('Begin!')
    
    while True:
        if collectedNum == dataSetSize:
            break
        frame = camera.read()
        #plt.imshow(frame)
        image_widget.value = bgr8_to_jpeg(frame)
        if loopcounter % (FPScollect * SEC_delay) == 0:
            # write raw images to file 
            captured_frame.value = bgr8_to_jpeg(frame)
            dH.write_raw_frame(path_raw, frame, collectedNum)
            collectedNum += 1
        loopcounter += 1
print('Done!')

Done!


## Data Labeling

An I oop... data is labeled on aonther PC because some libraries are broke

## Training Data Build

In [8]:

REBUILD = False


path_labeled = os.getcwd() + '/labeled_datasets'
data = os.scandir(path_labeled)


#fig = plt.figure()
#ax = fig.add_subplot(1,1,1)
#plt.axis("off")

fill = np.zeros((224,224,2))

if REBUILD:
    training_data = []
    for i in data:
        img = mpimg.imread(path_labeled +'/'+ i.name)
        [imgnum, x, y] = dH.label_parser(path_labeled +'/'+ i.name)
        #print(f'{imgnum},{x},{y}')
        #imgplot = plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        training_data.append([np.array(img),np.array([x,y])])
    np.random.shuffle(training_data)
    np.save('training_data.npy', training_data)




## Model Training

In [9]:
optimizer = optim.Adam(model.parameters(), lr = 0.0001)
loss_func = nn.MSELoss()

TRAIN = True

EPOCHS = 50
BATCH_SIZE = 10

if TRAIN:
    training_data = np.load('training_data.npy', allow_pickle=True)
    
    #np.random.shuffle(training_data)
    training_img = torch.Tensor([i[0] for i in training_data]).view(-1,3,224,224).to(device)
    training_label = torch.Tensor([i[1] for i in training_data]).view(-1,2).to(device)
        
    ## scale our images between 0 and 1
    training_img = training_img/255.0
    
    for epoch in range(EPOCHS):
        
        for i in range(0, len(training_data), BATCH_SIZE):
            model.zero_grad()
            output = model(training_img[i:i+BATCH_SIZE].view(-1,3,224,224))
            last_label = training_label[i:i+BATCH_SIZE]
            loss = loss_func(output,training_label[i:i+BATCH_SIZE].view(-1,2))
            loss.backward()
            optimizer.step()
        print(last_label)
        print(output)
        print(loss)

tensor([[111., 189.],
        [145.,  76.],
        [ 37.,  70.],
        [ 30.,  54.],
        [114.,  91.],
        [ 64.,  94.]], device='cuda:0')
tensor([[-0.7469,  1.0476],
        [ 0.6950, -0.8655],
        [ 0.1944,  0.2387],
        [-0.2144,  0.7538],
        [-0.6824, -0.4898],
        [-0.0222,  0.8711]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor(9900.7783, device='cuda:0', grad_fn=<MseLossBackward>)
tensor([[111., 189.],
        [145.,  76.],
        [ 37.,  70.],
        [ 30.,  54.],
        [114.,  91.],
        [ 64.,  94.]], device='cuda:0')
tensor([[-1.0070,  5.7166],
        [ 3.4947, -2.3535],
        [-0.6161, -0.2603],
        [-0.6563,  0.3945],
        [ 0.2793, -0.1201],
        [ 0.5109,  1.5994]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor(9689.1846, device='cuda:0', grad_fn=<MseLossBackward>)
tensor([[111., 189.],
        [145.,  76.],
        [ 37.,  70.],
        [ 30.,  54.],
        [114.,  91.],
        [ 64.,  94.]], device='cuda:0')
tens